# Generic

This is mostly copied from the previous notebook

In [67]:
import math
import base64
import binascii
import numpy as np
import matplotlib.pyplot as plt

def to_bytes(d, format):
    if isinstance(d, (bytes, bytearray)):
        return d
    elif format == 'hex':
        return bytes(bytearray.fromhex(d))
    elif format == 'base64':
        return base64.b64decode(d)
    elif format == 'str' or format == 'bytes':
        return d.encode()
    elif format == 'int':
        return to_bytes(hex(d)[2:], 'hex')

def bytes_to(b, format):
    if not isinstance(b, (bytes, bytearray)):
        return b
    elif format == 'hex':
        return binascii.hexlify(b).decode()
    elif format == 'base64':
        return base64.b64encode(b).decode()
    elif format == 'str':
        return b.decode()
    elif format == 'bytes':
        return b
    elif format == 'int':
        return int(bytes_to(b, 'hex')[2:], 16)
    
def draw_bytestring(bs):
    """Represent a byte string as a matrix of color
    """
    tmp = bs + bytes(math.ceil(math.sqrt(len(bs)))**2 - len(bs))
    tmp = [int(b) for b in tmp]
    m = [[tmp[i:i+int(math.sqrt(len(tmp)))] for i in range(0, len(tmp), int(math.sqrt(len(tmp))))]]
    matrix = np.matrix(m[0])
    fig = plt.figure()
    ax = fig.add_subplot(1,1,1)
    ax.set_aspect('equal')
    plt.imshow(matrix, interpolation='nearest', cmap=plt.cm.binary)
    plt.colorbar()
    plt.show()

## Helpers

Copied mostly from the previous notebook

In [3]:
import re
import string

def count_letter_freq(text):
    """Count the frequence of each letter in a string
    Return a dict
    """
    letters = {}
    total = 0
    for l in text:
        letters[l.lower()] = letters.get(l.lower(), 0) + 1
        total += 1
    return {l:s/total for l,s in letters.items()}

def hamming_distance(b1, b2):
    distance = 0
    for char1, char2 in zip(b1,b2):
        distance += bin(char1 ^ char2).count('1')
    return distance
assert hamming_distance(b'this is a test', b'wokka wokka!!!') == 37

def count_upper_lower_signs(text):
    """Count the amount of uppercase letters, lowercase letters and other signs
    """
    counter = {'upper': 0, 'lower': 0, 'signs': 0}
    for c in text:
        if c in string.ascii_lowercase:
            counter['lower'] += 1
        elif c in string.ascii_uppercase:
            counter['upper'] += 1
        else:
            counter['signs'] += 1
    return counter

NON_PRINTABLE_REGEX = "[^\w " + string.printable + "\n]"
def is_encodable(bs):
    try:
        return bs.decode()
    except UnicodeDecodeError:
        return False
def is_text(s):
    s_test = re.sub(NON_PRINTABLE_REGEX, "", s.strip())
    return s_test == s.strip()
def is_english(text):
    """Simple scoring function that counts probabilities of caracters and tries
    to match them with the probabilities of the english language"""
    if isinstance(text, (bytes, bytearray)):
        s = is_encodable(text)
    else:
        s = text
    if not s:
        return 0
    freqs = ['etaoin shrdlcumwfgypbvkjxqz', [12.702,9.056,8.167,7.507,6.966,6.749,6.4,6.327,6.094,5.987,4.253,4.025,2.782,2.758,2.406,2.360,2.228,2.015,1.974,1.929,1.492,0.978,0.772,0.153,0.150,0.095,0.074]]
    nominal_scores = {l:s for l,s in zip(freqs[0], freqs[1])}
    text_freqs = count_letter_freq(s)
    score = 0
    for letter, freq in text_freqs.items():
        if letter.lower() in nominal_scores:
            score += nominal_scores[letter.lower()]
        else:
            score -= 1
    return max(score / len(text), 0)

## Crypto 101

This is mostly copied from the previous notebook

In [4]:
import math
import itertools
from Crypto.Cipher import AES


def pad(text, blocksize=16):
    padsize = blocksize - (len(text) % blocksize)
    if padsize == 0:
        padsize = blocksize
    return text + bytes([padsize]*padsize)
assert pad(b"YELLOW SUBMARINE", blocksize=20) == b'YELLOW SUBMARINE\x04\x04\x04\x04'

def unpad(text):
    """Validate padding and return unpadded value"""
    assert text[-text[-1]:] == bytes([text[-1]]*text[-1])
    return text[:-text[-1]]
assert unpad(pad(b"this is random text", blocksize=7)) == b"this is random text"

import math
import itertools

def xor(d1, d2, format):
    b1,b2 = to_bytes(d1, format), to_bytes(d2, format)
    return bytes_to(bytes(char1 ^ char2 for char1,char2 in zip(b1,b2)), format)

def repeating_xor(s, key, format):
    bs = to_bytes(s, format)
    bkey = bytearray(itertools.islice(itertools.cycle(to_bytes(key, format)), len(bs)))
    return xor(bs, bkey, format)

def guess_repeating_xor_keysize(text, count=3, minsize=2, maxsize=40):
    """Try to guess the size of a repeating XOR key from a given ciphertext.
    Return top <count> guesses.
    """
    values = {}
    for keysize in range(minsize, maxsize+1):
        # We use 4 times the keysize in order to get a better reading
        values[keysize] = hamming_distance(text[:keysize*4], text[keysize*4:2*keysize*4]) / (keysize*4)
    values = [(keysize, value) for keysize, value in values.items()]
    values.sort(key=lambda x:x[1]) # The best guess is the lowest hamming distance
    return [keysize for keysize,_ in values[:count]]
    

def crack_single_char_xor(text):
    """Try to crack a cipher as a single character XOR of english text
    """
    bestguess = {'key': 0, 'confidence': -len(text), 'cleartext': ''}
    for i in range(256):
        try:
            score = is_english(xor(text, bytes([i]*len(text)), 'bytes'))
            if score > bestguess['confidence']:
                bestguess['key'] = bytes([i])
                bestguess['confidence'] = score
                bestguess['cleartext'] = xor(text, bytes([i]*len(text)), 'str')
        except UnicodeDecodeError:
            pass
    return bestguess

def crack_repeating_xor(text, keysize):
    """Try to crack a cipher as a repeating key XOR, with a given keysize
    """
    # Split into blocks of size keysize
    blocks = [text[i:i+keysize] for i in range(0, len(text), keysize)]
    
    # Now, refactor blocks to be all encrypted with the same caracter
    # (every n-th caracter of a block is encrypted with the same n-th caracter of the key)
    transposed_blocks = [bytes(block[n] for block in blocks if len(block) > n) for n in range(keysize)]
    
    # Solve every transposed block for single-caracter xor
    cleartext_transposed_blocks = [crack_single_char_xor(block) for block in transposed_blocks]
    
    # Rebuild and return cleartext
    #key = bytes(block['key'][0] for block in cleartext_transposed_blocks)
    key = b''.join(block['key'] for block in cleartext_transposed_blocks)
    cleartext = repeating_xor(text, key, 'str')
    confidence = sum(block['confidence'] for block in cleartext_transposed_blocks) / len(cleartext_transposed_blocks)
    
    return {'key': key, 'cleartext':cleartext, 'confidence':confidence}

def decrypt_aes_ecb(cipher, key, IV=None, use_padding=True):
    """No magic here
    (The IV is not actually used)"""
    aes = AES.new(key, AES.MODE_ECB)
    if use_padding:
        return unpad(aes.decrypt(cipher))
    else:
        return aes.decrypt(cipher)
def encrypt_aes_ecb(clear, key, IV=None, use_padding=True):
    """No magic here
    (The IV is not actually used)"""
    aes = AES.new(key, AES.MODE_ECB)
    if use_padding:
        clear = pad(clear)
    return aes.encrypt(clear)
assert decrypt_aes_ecb(encrypt_aes_ecb(b'test', 'YELLOW SUBMARINE'), 'YELLOW SUBMARINE') == b'test'

def encrypt_aes_cbc(clear, key, IV=None):
    """CBC using the ECB mode of above
    """
    if IV is None:
        IV=b'\x00'*16
    clear = pad(clear)
    prev_block = IV
    cipher = b''
    for blockstart in range(0, len(clear), 16):
        block = clear[blockstart:blockstart+16]
        cipher_block = encrypt_aes_ecb(xor(prev_block, block, 'bytes'), key, use_padding=False)
        prev_block = cipher_block
        cipher += cipher_block
    return cipher

def decrypt_aes_cbc(cipher, key, IV=None):
    """CBC using the ECB mode of above
    """
    if IV is None:
        IV=b'\x00'*16
    clear = b''
    prev_block = IV
    for blockstart in range(0, len(cipher), 16):
        block = cipher[blockstart:blockstart+16]
        tmp = decrypt_aes_ecb(block, key, use_padding=False)
        clear_block = xor(prev_block, tmp, 'bytes')
        prev_block = cipher[blockstart:blockstart+16]
        clear += clear_block
    return unpad(clear)
assert decrypt_aes_cbc(encrypt_aes_cbc(b'test'*200, b'YELLOW SUBMARINE'), b'YELLOW SUBMARINE') == b'test'*200


def aes_ctr_generator(key, nonce=0):
    """Generate a keystream using AES-CTR.
    """
    while True:
        counter = nonce.to_bytes(9, 'big') + b'\x00'*7
        yield encrypt_aes_ecb(counter, key, use_padding=False)
        nonce += 1
        
def encrypt_aes_ctr(key, cleartext, nonce=0):
    """Encrypt some text with AEs-CTR.
    """
    cipher = b''
    keystream = aes_ctr_generator(key, nonce=nonce)
    for i in range(0, len(cleartext), 16):
        block = cleartext[i:i+16]
        cipher += xor(block, next(keystream), 'bytes')
    return cipher

decrypt_aes_ctr = encrypt_aes_ctr # Such elegance !

## Crypto 102

In [5]:
def padding_oracle_attack(cipher, oracle, iv, blocksize=16):
    # The decryption is made block by block in order and byte per byte in reverse order
    # The first block being the IV, it is not decrypted
    
    # Note : this is a statistical attack and may end up looping forever.
    # You should watch out for the 'Cracking block' messages
    
    blocks = [iv] + [cipher[i:i+blocksize] for i in range(0, len(cipher), blocksize)]
    cleartext = b''
    for blockidx in range(1, len(blocks)):
        print('Cracking block', blockidx)
        suffix = b''
        block = blocks[blockidx]
        prevblock = blocks[blockidx-1]
        clearblock = b''
        for i in range(len(block)-1, -1, -1):
            # Make random changes to ith byte of block[n-1]
            # If the result has been accepted, then we know
            # we have a padding of size i

            testbyte = bytes([random.randint(0, 255)])
            testblock = prevblock[:-len(suffix)-1] + testbyte + suffix
            if blockidx == 1:
                iv = testblock
            else:
                iv = blocks[0]
            while not oracle(b''.join(blocks[1:blockidx-1]) + testblock + blocks[blockidx], iv):
                testbyte = bytes([random.randint(0, 255)])
                testblock = prevblock[:-len(suffix)-1] + testbyte + suffix

            # We passed \o/ !
            expected_padding_byte = bytes([16-i])
            clearblock = xor(xor(expected_padding_byte, testbyte, 'bytes'), bytes([prevblock[i]]), 'bytes') + clearblock

            # Update stuff
            next_expected_padding_byte = bytes([16-i+1])
            old_suffix = suffix
            suffix = xor(xor(testbyte, expected_padding_byte, 'bytes'), next_expected_padding_byte, 'bytes')
            suffix += xor(xor(old_suffix, expected_padding_byte*len(old_suffix), 'bytes'), next_expected_padding_byte*len(old_suffix), 'bytes')

        # Return cleartext
        cleartext += clearblock

    return cleartext

A generic class for Mersenne Twisters.
Also, tools allowing us to do stuff around the MT19937 RNG.

In [6]:
import random

class MersenneTwister:
    def __init__(self, w, n, m, r, a, b, c, s, t, u, d, l, f):
        assert w > 2
        assert 1 <= m < n
        assert 0 <= r < w
        self.w, self.n, self.m, self.r = w, n, m, r
        self.a, self.b, self.c, self.s = a, b, c, s
        self.t, self.u, self.d, self.l = t, u, d, l
        self.f = f
        self.index = 0 # Keep track of how many numbers we have generated, modulo n
        self.real_index = 0 # Actual counter
        
    def _get_word(self, i):
        return int(i & (2**self.w - 1))
    
    def _get_low(self, i):
        return self._get_word(int(i & (2**self.r - 1)))
    
    def _get_hi(self, i):
        return self._get_word(int(i & ((2**(self.w - self.r)-1)*2**self.r)))
    
    def seed(self, x0):
        wx0 = self._get_word(x0) # Keep only lowest w bits
        self._buffer = [wx0]
        self.index = self.n
        for i in range(1, self.n):
            self._buffer.append(
                self._get_word(self.f * \
                                (self._buffer[i-1] ^ (self._buffer[i-1] >> (self.w - 2))) + i)
            )
        return True
    
    def generate(self):
        if self.index >= self.n:
            self.twist()
            
        y = self._buffer[self.index]
        y = y ^ ((y >> self.u) & self.d)
        y = y ^ ((y << self.s) & self.b)
        y = y ^ ((y << self.t) & self.c)
        z = y ^ (y >> self.l)
        
        self.index += 1
        self.real_index += 1
        return self._get_word(z)
    
    def twist(self):
        for i in range(self.n):
            y = self._get_hi(self._buffer[i]) + self._get_low(self._buffer[(i+1) % self.n])
            self._buffer[i] = self._buffer[(i+self.m) % self.n] ^ (y >> 1)
            if y % 2 != 0:
                self._buffer[i] = self._buffer[i] ^ self.a
        self.index = 0
        
            
def Mersenne19937(i):
    m = MersenneTwister(w=32, n=624, m = 397, r=31,
                    a=0x9908B0DF, u=11, d=0xFFFFFFFF,
                    s=7, b=0x9D2C5680, t=15,
                    c=0xEFC60000, l=18, f=1812433253)
    m.seed(i)
    return m

def undo_rightshift_mask(z, s, m, wordlength=32):
    """Find a where z = a ^ [(a >> s) & m]
    """
    binary_a = ''
    binary_z = bin(z)[2:]
    binary_m = bin(m)[2:]
    # Pad m and z representations to wordlength :
    binary_z = '0' * max(wordlength - len(binary_z), 0) + binary_z
    binary_m = '0' * max(wordlength - len(binary_m), 0) + binary_m
    for i in range(s): # The first bits are easy
        binary_a = binary_a + binary_z[i]
    for i in range(s, wordlength): # The rest may required alreayd known bits
        if binary_m[i] == '1':
            binary_a = binary_a + str(int(binary_z[i]) ^ int(binary_a[i-s])) # a[i-s] is known !
        else:
            binary_a = binary_a + binary_z[i]
    return int(binary_a, 2)
        

def undo_leftshift_mask(z, s, m, wordlength=32):
    """Find a where z = a ^ [(a << s) & m]
    """
    binary_a = ''
    binary_z = bin(z)[2:]
    binary_m = bin(m)[2:]
    # Pad m and z representations to wordlength :
    binary_z = '0' * max(wordlength - len(binary_z), 0) + binary_z
    binary_m = '0' * max(wordlength - len(binary_m), 0) + binary_m
    for i in range(s): # The first bits are easy
        binary_a = binary_z[-i-1] + binary_a
    for i in range(s, wordlength): # The rest may required alreayd known bits
        if binary_m[-i-1] == '1':
            binary_a = str(int(binary_z[-i-1]) ^ int(binary_a[-i-1+s])) + binary_a # a[-i-1+s] is known !
        else:
            binary_a = binary_z[-i-1] + binary_a
    return int(binary_a, 2)

# Test our reversers
y = random.randint(1, 2**32)
m = random.randint(1, 2**32)
s = random.randint(1, 32)
z = y ^ y >> s & m
z2 = y ^ y << s & m
assert y == undo_rightshift_mask(z, s, m)
assert y == undo_leftshift_mask(z2, s, m)

def untemper(i):
    """Assume MT19937 RNG"""
    i = undo_rightshift_mask(i, 18, 0xFFFFFFFF)
    i = undo_leftshift_mask(i, 15, 4022730752)
    i = undo_leftshift_mask(i, 7, 2636928640)
    return undo_rightshift_mask(i, 11, 0xFFFFFFFF)

# A MT19937 generator is determined entirely by its 624 state integers !
def clone_rng(rng):
    """Assume MT19937 RNG"""
    _state = []
    for i in range(624):
        _state.append(untemper(rng.generate()))
    clone = Mersenne19937(0)
    clone._buffer = _state
    clone.index = 624 # We just generated 624 numbers with the old RNG
    return clone

### Crypto 103

In [7]:
def mt19937_ctr_generator(key):
    # Ensure the key is 16bit
    if isinstance(key, str):
        assert len(key) >= 2
        key = ord(key[1]) * 0xFF + ord(key[0])
    elif isinstance(key, bytes):
        assert len(key) >= 2
        key = key[1] * 0xFF + key[0]
    key16bit = key & 0xFFFF
    rng = Mersenne19937(key16bit)
    while True:
        i = rng.generate()
        yield bytes([i & 0xFF])
        yield bytes([(i & 0xFF00) >> 8])
        yield bytes([(i & 0xFF0000) >> 16])
        yield bytes([(i & 0xFF000000) >> 24])
        
def encrypt_mt19937_ctr(key, cleartext):
    keystream = mt19937_ctr_generator(key)
    cipher = b''
    for c in cleartext:
        n = next(keystream)
        cipher += xor(bytes([c]), next(keystream), 'bytes')
    return cipher

decrypt_mt19937_ctr = encrypt_mt19937_ctr # Such elegance !

assert decrypt_mt19937_ctr('key', encrypt_mt19937_ctr('key', b'this is a long cleartext')) == b'this is a long cleartext'

# Cryptopals notebook - part 2

## Set 3

### Exercice 17

Start with a few context functions

In [5]:
import os, random

def producer():
    # I'll assume it's not one key per random string
    key = b'u\x9b(U\xdb\\\xe9\xe3B\x1dN&\xb3\xb6\xb7\xa0'
    IV = os.urandom(16)
    s =  random.choice(['MDAwMDAwTm93IHRoYXQgdGhlIHBhcnR5IGlzIGp1bXBpbmc=',
                          'MDAwMDAxV2l0aCB0aGUgYmFzcyBraWNrZWQgaW4gYW5kIHRoZSBWZWdhJ3MgYXJlIHB1bXBpbic=',
                          'MDAwMDAyUXVpY2sgdG8gdGhlIHBvaW50LCB0byB0aGUgcG9pbnQsIG5vIGZha2luZw==',
                          'MDAwMDAzQ29va2luZyBNQydzIGxpa2UgYSBwb3VuZCBvZiBiYWNvbg==',
                          'MDAwMDA0QnVybmluZyAnZW0sIGlmIHlvdSBhaW4ndCBxdWljayBhbmQgbmltYmxl',
                          'MDAwMDA1SSBnbyBjcmF6eSB3aGVuIEkgaGVhciBhIGN5bWJhbA==',
                          'MDAwMDA2QW5kIGEgaGlnaCBoYXQgd2l0aCBhIHNvdXBlZCB1cCB0ZW1wbw==',
                          'MDAwMDA3SSdtIG9uIGEgcm9sbCwgaXQncyB0aW1lIHRvIGdvIHNvbG8=',
                          'MDAwMDA4b2xsaW4nIGluIG15IGZpdmUgcG9pbnQgb2g=',
                          'MDAwMDA5aXRoIG15IHJhZy10b3AgZG93biBzbyBteSBoYWlyIGNhbiBibG93',])
    return encrypt_aes_cbc(to_bytes(s, 'base64'), key, IV=IV), IV

def consumer(c, IV):
    key = b'u\x9b(U\xdb\\\xe9\xe3B\x1dN&\xb3\xb6\xb7\xa0'
    try:
        decrypt_aes_cbc(c, key=key, IV=IV)
        return True
    except AssertionError as e:
        # This means a padding error has been detected
        return False
    
assert consumer(*producer())

In [8]:
# These two functions should allow us to break the crypto
cipher, IV = producer()
print(padding_oracle_attack(oracle=consumer, iv=IV, cipher=cipher))

Cracking block 1
Cracking block 2
Cracking block 3
b'000005I go crazy when I hear a cymbal\x0b\x0b\x0b\x0b\x0b\x0b\x0b\x0b\x0b\x0b\x0b'


### Exercice 18

In [9]:
cipher = to_bytes('L77na/nrFsKvynd6HzOoG7GHTLXsTVu9qvY/2syLXzhPweyyMTJULu/6/kXX0KSvoOLSFQ==', 'base64')
key = b'YELLOW SUBMARINE'
print(decrypt_aes_ctr(key, cipher))

b"Yo, VIP Let's kick it Ice, Ice, baby Ice, Ice, baby "


### Exercice 19

This exercice will not be fully automated. Apparently, this method is suboptimal.

In [6]:
# Generate a series of ciphers using the same nonce and key everytime

key = b'6W\x1a\\h\xcbAb\xdfV!i\xbdo\xfc#'
ciphers = ['SSBoYXZlIG1ldCB0aGVtIGF0IGNsb3NlIG9mIGRheQ==',
'Q29taW5nIHdpdGggdml2aWQgZmFjZXM=',
'RnJvbSBjb3VudGVyIG9yIGRlc2sgYW1vbmcgZ3JleQ==',
'RWlnaHRlZW50aC1jZW50dXJ5IGhvdXNlcy4=',
'SSBoYXZlIHBhc3NlZCB3aXRoIGEgbm9kIG9mIHRoZSBoZWFk',
'T3IgcG9saXRlIG1lYW5pbmdsZXNzIHdvcmRzLA==',
'T3IgaGF2ZSBsaW5nZXJlZCBhd2hpbGUgYW5kIHNhaWQ=',
'UG9saXRlIG1lYW5pbmdsZXNzIHdvcmRzLA==',
'QW5kIHRob3VnaHQgYmVmb3JlIEkgaGFkIGRvbmU=',
'T2YgYSBtb2NraW5nIHRhbGUgb3IgYSBnaWJl',
'VG8gcGxlYXNlIGEgY29tcGFuaW9u',
'QXJvdW5kIHRoZSBmaXJlIGF0IHRoZSBjbHViLA==',
'QmVpbmcgY2VydGFpbiB0aGF0IHRoZXkgYW5kIEk=',
'QnV0IGxpdmVkIHdoZXJlIG1vdGxleSBpcyB3b3JuOg==',
'QWxsIGNoYW5nZWQsIGNoYW5nZWQgdXR0ZXJseTo=',
'QSB0ZXJyaWJsZSBiZWF1dHkgaXMgYm9ybi4=',
'VGhhdCB3b21hbidzIGRheXMgd2VyZSBzcGVudA==',
'SW4gaWdub3JhbnQgZ29vZCB3aWxsLA==',
'SGVyIG5pZ2h0cyBpbiBhcmd1bWVudA==',
'VW50aWwgaGVyIHZvaWNlIGdyZXcgc2hyaWxsLg==',
'V2hhdCB2b2ljZSBtb3JlIHN3ZWV0IHRoYW4gaGVycw==',
'V2hlbiB5b3VuZyBhbmQgYmVhdXRpZnVsLA==',
'U2hlIHJvZGUgdG8gaGFycmllcnM/',
'VGhpcyBtYW4gaGFkIGtlcHQgYSBzY2hvb2w=',
'QW5kIHJvZGUgb3VyIHdpbmdlZCBob3JzZS4=',
'VGhpcyBvdGhlciBoaXMgaGVscGVyIGFuZCBmcmllbmQ=',
'V2FzIGNvbWluZyBpbnRvIGhpcyBmb3JjZTs=',
'SGUgbWlnaHQgaGF2ZSB3b24gZmFtZSBpbiB0aGUgZW5kLA==',
'U28gc2Vuc2l0aXZlIGhpcyBuYXR1cmUgc2VlbWVkLA==',
'U28gZGFyaW5nIGFuZCBzd2VldCBoaXMgdGhvdWdodC4=',
'VGhpcyBvdGhlciBtYW4gSSBoYWQgZHJlYW1lZA==',
'QSBkcnVua2VuLCB2YWluLWdsb3Jpb3VzIGxvdXQu',
'SGUgaGFkIGRvbmUgbW9zdCBiaXR0ZXIgd3Jvbmc=',
'VG8gc29tZSB3aG8gYXJlIG5lYXIgbXkgaGVhcnQs',
'WWV0IEkgbnVtYmVyIGhpbSBpbiB0aGUgc29uZzs=',
'SGUsIHRvbywgaGFzIHJlc2lnbmVkIGhpcyBwYXJ0',
'SW4gdGhlIGNhc3VhbCBjb21lZHk7',
'SGUsIHRvbywgaGFzIGJlZW4gY2hhbmdlZCBpbiBoaXMgdHVybiw=',
'VHJhbnNmb3JtZWQgdXR0ZXJseTo=',
'QSB0ZXJyaWJsZSBiZWF1dHkgaXMgYm9ybi4=]',]
ciphers = [encrypt_aes_ctr(key, to_bytes(c, 'base64')) for c in ciphers]

Now, let's break'em. Fixed nonce CTR is basically a fixed key XOR with a long key, so statistical analysis applies. Or brute force even. There are only 255 possibilities for each letter...

In [41]:
import string
import itertools

def get_ascii_xor_key(s):
    """Using a very simplistic heuristic, try to find 
    a possible repeating, 1-char XOR key for a given string
    that ensures that the target is in target ASCII english caracters
    """
    possibilities = []
    for i in range(256):
        result = xor(s, bytes([i]*len(s)), 'bytes')
        try:
            assert all(l in string.ascii_letters + ' \',.-;:?!' for l in result.decode('ascii'))
            possibilities.append(bytes([i]))
        except:
            pass
    return possibilities
        

# First, bruteforce and check if every letter is in the desired target space
# We build a set of possible keys based on that simple criteria.
cleartexts = []
keys = []
maxlen = max(len(c) for c in ciphers)
for idx in range(maxlen):
    hline = bytes([c[idx] for c in ciphers if idx<len(c)])
    possibilities = get_ascii_xor_key(hline)
    if possibilities != []:
        keys.append(possibilities)
    else:
        keys.append(range(256))

# We now have a set of keys :
keyspace = 1
for idx, l in enumerate([len(k) for k in keys]):
    if l<3:
        print('Byte', idx, 'has only', l, 'possibilities remaining')
    keyspace *= l
print('We found %s possible keys' % keyspace)

# Up until byte 31, its pretty good, and there are only 2048 keys left.
# Let's base it around these 2048 and decode the remaining 6 bytes by hand if necessary.
short_keys = [k*(len(k)<3) + [k[0]]*(len(k)>2) for k in keys]
keyspace = [b''.join(k) for k in itertools.product(*short_keys)]

Byte 0 has only 2 possibilities remaining
Byte 1 has only 2 possibilities remaining
Byte 2 has only 1 possibilities remaining
Byte 3 has only 2 possibilities remaining
Byte 4 has only 1 possibilities remaining
Byte 5 has only 2 possibilities remaining
Byte 6 has only 1 possibilities remaining
Byte 7 has only 2 possibilities remaining
Byte 8 has only 1 possibilities remaining
Byte 9 has only 1 possibilities remaining
Byte 10 has only 1 possibilities remaining
Byte 11 has only 1 possibilities remaining
Byte 12 has only 1 possibilities remaining
Byte 13 has only 1 possibilities remaining
Byte 14 has only 2 possibilities remaining
Byte 15 has only 1 possibilities remaining
Byte 16 has only 1 possibilities remaining
Byte 17 has only 1 possibilities remaining
Byte 18 has only 1 possibilities remaining
Byte 19 has only 1 possibilities remaining
Byte 20 has only 1 possibilities remaining
Byte 21 has only 1 possibilities remaining
Byte 22 has only 1 possibilities remaining
Byte 23 has only 2 po

In [54]:
# Score everey key and find max
import tqdm

def decrypt_ciphers_and_concatenate(key, ciphers):
    clears = b''
    for c in ciphers:
        clears += xor(key, c, 'bytes')
    return bytes_to(clears, 'str')

scored_keys = []
max_key = keyspace[0]
max_score = is_english(decrypt_ciphers_and_concatenate(keyspace[0], ciphers))
max_idx = 0
scored_keys.append((max_score, keyspace[0]))
for idx, k in tqdm.tqdm(enumerate(keyspace[1:])):
    score = is_english(decrypt_ciphers_and_concatenate(k, ciphers))
    scored_keys.append((score, k))
    if score > max_score:
        max_score = score
        max_key = k
        max_idx = idx

scored_keys.sort(key=lambda x:x[0], reverse=True)
print('Max key', max_key, 'with score', max_score)
print('Result :', [xor(scored_keys[0][1], c, 'bytes') for c in ciphers])

Max key b"\x96\xb0\x84\x18l\xa1\x1c/\xd7\xdd\xd1\x8f\xed4\x9a;\x1d\xc7O\x9f\x0b\x93\x06\xb7':9C\x0b\xd1J*2\xaf'\x00\xa1\x00" with score 0.08892121754700091
Result : [b'i hfve met thej at close of dax', b'comnng with viqid faces', b'froj counter ou desk among grex', b'eigoteenth-censury houses.', b'i hfve passed pith a nod of thdih..E', b'or wolite meannngless words,', b'or oave lingerbd awhile and sah-', b'polnte meaningkess words,', b"and'thought beaore I had done", b'of f mocking tfle or a gibe', b'to wlease a cojpanion', b'arornd the firb at the club,', b'beiig certain shat they and I', b"but'lived wherb motley is worn;", b"all'changed, coanged utterly:", b'a tbrrible bearty is born.', b"thas woman's dfys were spent", b'in ngnorant gohd will,', b"her'nights in frgument", b'untnl her voicb grew shrill.', b'whas voice morb sweet than herr', b"whei young and'beautiful,", b"she'rode to hauriers?", b'thit man had kbpt a school', b"and'rode our wnnged horse.", b"thit other his'helper and 

In [73]:
# By hand, we can find a few cleartexts and deduce the final key
cleartexts = {0:b'I have met them at close of day'}
finalkey = xor(cleartexts[0], ciphers[0], 'bytes')
finalkey += scored_keys[0][1][len(finalkey):]

# Iterate by finding some cleartexts by hand
print('Result :', [(idx,xor(finalkey, c, 'bytes')) for idx,c in enumerate(ciphers) if len(c)>len(cleartexts[0])])

cleartexts[27] = b'He might have won fame in the end\n'
finalkey = xor(cleartexts[27], ciphers[27], 'bytes')
finalkey += scored_keys[27][1][len(finalkey):]

# Iterate by finding some cleartexts by hand
print('Result :', [(idx,xor(finalkey, c, 'bytes')) for idx,c in enumerate(ciphers) if len(c)>len(cleartexts[27])])

cleartexts[4] = b'I have passed with a nod of the head'
finalkey = xor(cleartexts[4], ciphers[4], 'bytes')
finalkey += scored_keys[4][1][len(finalkey):]

# Iterate by finding some cleartexts by hand
print('Result :', [(idx,xor(finalkey, c, 'bytes')) for idx,c in enumerate(ciphers) if len(c)>len(cleartexts[4])-2])

cleartexts[37] = b'He, too, has been changed in his turn\n'
finalkey = xor(cleartexts[37], ciphers[37], 'bytes')
finalkey += scored_keys[37][1][len(finalkey):]

# Final cleartexts :
print('Result :', [(idx,xor(finalkey, c, 'bytes')) for idx,c in enumerate(ciphers)])

Result : [(4, b'I have passed with a nod of theih..E'), (6, b'Or have lingered awhile and sai-'), (25, b'This other his helper and frien-'), (27, b"He might have won fame in the e'dg"), (29, b'So daring and sweet his thoughtg'), (37, b'He, too, has been changed in hi: ?:S.d')]
Result : [(4, b'I have passed with a nod of the hC.E'), (37, b'He, too, has been changed in his R:S.d')]
Result : [(4, b'I have passed with a nod of the head'), (37, b'He, too, has been changed in his tur.d')]
Result : [(0, b'I have met them at close of day'), (1, b'Coming with vivid faces'), (2, b'From counter or desk among grey'), (3, b'Eighteenth-century houses.'), (4, b'I have passed with a nod of the head'), (5, b'Or polite meaningless words,'), (6, b'Or have lingered awhile and said'), (7, b'Polite meaningless words,'), (8, b'And thought before I had done'), (9, b'Of a mocking tale or a gibe'), (10, b'To please a companion'), (11, b'Around the fire at the club,'), (12, b'Being certain that they and I'), (13

### Exercice 20

Same weakness - but with repeating XOR cracking method

In [79]:
key = b'^!1w \x8e$\xe0,M\x01-\xe0\xe5(\xa1'
with open('20.txt') as f:
    ciphers = [to_bytes(l, 'base64') for l in f.readlines()]
ciphers = [encrypt_aes_ctr(key, to_bytes(c, 'base64')) for c in ciphers]    

In [110]:
# Repeating XOR attack on same prefix length

l = min(len(c) for c in ciphers)
crack_repeating_xor(text=b''.join(c[:l] for c in ciphers), keysize=l)

# The result is pretty good. We are not going to push it, its the same as above after that.

{'cleartext': 'N\'m rated "R"...this is a warning, ya better void / PDuz I came back to attack others in spite- / Strike lEut don\'t be afraid in the dark, in a park / Not a sc^a tremble like a alcoholic, muscles tighten up / WhaTuddenly you feel like your in a horror flick / You gJusic\'s the clue, when I come your warned / ApocalypsOaven\'t you ever heard of a MC-murderer? / This is thCeath wish, so come on, step to this / Hysterical ideAriday the thirteenth, walking down Elm Street / You Shis is off limits, so your visions are blurry / All Serror in the styles, never error-files / Indeed I\'m Aor those that oppose to be level or next to this / IPorse than a nightmare, you don\'t have to sleep a winAlashbacks interfere, ya start to hear: / The R-A-K-IShen the beat is hysterical / That makes Eric go get Toon the lyrical format is superior / Faces of death JC\'s decaying, cuz they never stayed / The scene of aShe fiend of a rhyme on the mic that you know / It\'s Jelodies-unmakable, pat

### Exercice 21

Mersenne twister - reimplemented based on the wikipedia article

In [249]:
# --base class in Crypto 102--            

In [250]:
def _int32(x):
    # Get the 32 least significant bits.
    return int(0xFFFFFFFF & x)

class MT19937:

    def __init__(self, seed):
        # Initialize the index to 0
        self.index = 624
        self.mt = [0] * 624
        self.mt[0] = seed  # Initialize the initial state to the seed
        for i in range(1, 624):
            self.mt[i] = _int32(
                1812433253 * (self.mt[i - 1] ^ self.mt[i - 1] >> 30) + i)

    def extract_number(self):
        if self.index >= 624:
            self.twist()

        y = self.mt[self.index]

        # Right shift by 11 bits
        y = y ^ y >> 11
        # Shift y left by 7 and take the bitwise and of 2636928640
        y = y ^ y << 7 & 2636928640
        # Shift y left by 15 and take the bitwise and of y and 4022730752
        y = y ^ y << 15 & 4022730752
        # Right shift by 18 bits
        y = y ^ y >> 18

        self.index = self.index + 1

        return _int32(y)

    def twist(self):
        for i in range(624):
            # Get the most significant bit and add it to the less significant
            # bits of the next number
            y = _int32((self.mt[i] & 0x80000000) +
                       (self.mt[(i + 1) % 624] & 0x7fffffff))
            self.mt[i] = self.mt[(i + 397) % 624] ^ y >> 1

            if y % 2 != 0:
                self.mt[i] = self.mt[i] ^ 0x9908b0df
        self.index = 0
        #test

In [251]:
# Test using Wikipedia implementation as reference
import os
i = ord(os.urandom(1))
m1 = Mersenne19937(i)
m2 = MT19937(i)
for i in range(100):
    assert m1.generate() == m2.extract_number()

### Exercice 22

In [15]:
import random, time

# Routine - with a max time of 100s because, you know. Time.
def routine():
    time.sleep(random.randint(40, 100))
    seed = int(time.time())
    print('Seed :', seed)
    rng = Mersenne19937(seed)
    time.sleep(random.randint(1, 30))
    return rng.generate()

output = routine()

Seed : 1492444743


In [16]:
# One coffee later...
# Just bruteforce the seed. We now when it happened approximately, just test every possibility.
base = int(time.time())
for i in range(100000):
    seed = base -  i
    rng = Mersenne19937(seed)
    if rng.generate() == output:
        print('Seed was %s' % seed)
        break

Seed was 1492444743


### Exercice 23
The main exercice here is to reverse the tempering function of the Mersenne Twister. If a hash was applied to the output of the RNG, this would make it impossible to find the original state value, but it may also mess with the randomness and output distribution properties of the generator. Depends on the hash I guess.

In [321]:
# Demo of the tools that are defined in Crypto 102
m = Mersenne19937(153)
c = clone_rng(m)
for i in range(1000):
    assert c.generate() == m.generate()
    # Boom ! We got ourselves a clone !

### Exercice 24
Generate a Cipher stream based on the MT19937. Then crack it with the tools we just built.

In [8]:
# See Crypto 103.

In [49]:
import os

def cipher_generator(cleartext):
    key = b'\x00'  + os.urandom(1) # To avoid long bruteforce times, make it a small key
    print('Key :', bytes_to(key, 'hex'))
    prefix = os.urandom(random.randint(5,100))
    return encrypt_mt19937_ctr(key=key, cleartext=prefix + cleartext)

controlled_cipher = cipher_generator(b'A'*14)

Key : 0025


In [53]:
# 16 bit random key ! Bruteforce.
# To check the result, just generate enough numbers and check if we find our A's.
# Then we got our seed.
import tqdm
random_prefix = b'C' * (len(controlled_cipher)-14) # Just a filler string
controlled_suffix = b'A' * 14 # The actual string we are looking for
for i in tqdm.tqdm(range(0x10000)):
    mt = Mersenne19937(i)
    gnumbers = [mt.generate() for _ in range(len(controlled_cipher))]
    gbytes = map(lambda x: [0xFF & x,
                            0xFF & (x >> 8),
                            0xFF & (x >> 16),
                            0xFF & (x >> 24)], gnumbers)
    finalbytes = bytes(sum(gbytes, []))
    key = bytes([i & 0xFF00, i & 0xFF])
    check = encrypt_mt19937_ctr(key=key, cleartext=random_prefix + controlled_suffix)
    if check[-14:] == controlled_cipher[-14:]:
        print('Found the key :', bytes_to(key, 'hex'))
        break


  0%|          | 30/65536 [00:01<1:10:36, 15.46it/s]

  0%|          | 36/65536 [00:02<1:09:57, 15.61it/s]

Found the key : 0025


  0%|          | 36/65536 [00:22<11:09:31,  1.63it/s]

In [104]:
# Generate "random" password reset token
import time
def get_reset_token():
    t = int(time.time())
    print('Generating token on', t)
    mt = Mersenne19937(t)
    return hex(mt.generate())[2:]

In [105]:
get_reset_token()

Generating token on 1492447940


'dfef344b'

In [106]:
def check_current_time_token(token):
    # Get current time and check about 60s into the past
    t = int(time.time())
    for i in range(60):
        mt = Mersenne19937(t - i)
        check = hex(mt.generate())[2:]
        if check == token:
            print('Token was generated on time', t - i)
            return t - i

In [107]:
time.sleep(5)
check_current_time_token(get_reset_token())
time.sleep(2)
check_current_time_token(get_reset_token())
time.sleep(1)
check_current_time_token(get_reset_token())

Generating token on 1492447975
Token was generated on time 1492447975
Generating token on 1492447977
Token was generated on time 1492447977
Generating token on 1492447978
Token was generated on time 1492447978


1492447978